In [1]:
print("BEGIN DOWNLOAD")
!pip install torch --upgrade
print("DOWNLOADED PYTORCH")
!pip install torchvision --upgrade
print("DOWNLOADED PYTORCH-VISION")

BEGIN DOWNLOAD
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import csv
import xml.etree.ElementTree as ET
import json
import torchvision as tv
import torch as t
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import ssl
import gc
from PIL import Image
import shutil
import cv2
from torchvision.transforms import functional as F
import psutil
import warnings
warnings.filterwarnings('ignore') 
print("IMPORTED LIBRARIES")
ssl._create_default_https_context = ssl._create_unverified_context
if not os.path.exists('dataset'):
    os.mkdir('dataset')
    os.mkdir('train')
    os.mkdir('test')
    os.mkdir('dataset/drafter_00')
    print("CREATED FOLDERS")

IMPORTED LIBRARIES
CREATED FOLDERS


In [3]:
!cp -r /kaggle/input/cghd1152/* /kaggle/working/dataset

# Preprocessing

In [4]:
def read_labels(classes_json: str):
    with open(classes_json, "r", encoding="utf-8") as f:
        label_map = json.load(f)
    return label_map


def xml_to_csv(xml_file, csv_file, label_map):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    with open(csv_file, "w", newline="", encoding="utf-8") as csv_file:
        csv_writer = csv.writer(csv_file)
        headers = [
            "Label",
            "object",
            "xmin",
            "ymin",
            "xmax",
            "ymax",
            "rotation",
            "text",
        ]
        csv_writer.writerow(headers)
        for obj in root.findall(".//object"):
            object_name = obj.find("name").text
            label = label_map.get(object_name, "unknown")
            bndbox = obj.find("bndbox")
            xmin = bndbox.find("xmin").text
            ymin = bndbox.find("ymin").text
            xmax = bndbox.find("xmax").text
            ymax = bndbox.find("ymax").text
            rotation_element = bndbox.find("rotation")
            rotation = rotation_element.text if rotation_element is not None else ""
            text_element = obj.find("text")
            text = text_element.text if text_element is not None else ""
            row = [label, object_name, xmin, ymin, xmax, ymax, rotation, text]
            csv_writer.writerow(row)


def process_drafter_folders(label_map_file):
    label_map = read_labels(label_map_file)
    drafter_folders = [f"/kaggle/working/dataset/drafter_{i}" for i in range(26)]
    for drafter_folder in drafter_folders:
        annotations_folder = os.path.join(drafter_folder, "annotations")
        csv_output_folder = os.path.join(drafter_folder, "csvs")
        os.makedirs(csv_output_folder, exist_ok=True)
        for xml_file in os.listdir(annotations_folder):
            if xml_file.endswith(".xml"):
                xml_file_path = os.path.join(annotations_folder, xml_file)
                csv_file_path = os.path.join(
                    csv_output_folder, f"{os.path.splitext(xml_file)[0]}.csv"
                )

                xml_to_csv(xml_file_path, csv_file_path, label_map)

process_drafter_folders("/kaggle/working/dataset/classes.json")

In [5]:
print("CSV FILES CREATED")

CSV FILES CREATED


In [6]:
for i in range(1, 25):
    img_folder = f"/kaggle/working/dataset/drafter_{i}/images/"
    csv_folder = f"/kaggle/working/dataset/drafter_{i}/csvs/"

    for file in os.listdir(img_folder):
        if file.split('.')[0] + '.csv' in os.listdir(csv_folder):
            shutil.move(img_folder + file, "train/" + file.split(".")[0] + ".jpg")
            shutil.move(
                csv_folder + file.split(".")[0] + ".csv",
                "train/" + file.split(".")[0] + ".csv",
            )


for file in os.listdir("/kaggle/working/dataset/drafter_0/images/"):
    image = cv2.imread("/kaggle/working/dataset/drafter_0/images/" + file)
    cv2.imwrite(
        "/kaggle/working/dataset/drafter_00/" + file.split(".")[0] + ".jpg",
        image,
        [int(cv2.IMWRITE_JPEG_QUALITY), 100],
    )

img_folder = "/kaggle/working/dataset/drafter_00/"
csv_folder = "/kaggle/working/dataset/drafter_0/csvs/"

for file in os.listdir(img_folder):
    if file.split('.')[0] + '.csv' in os.listdir(csv_folder):
        shutil.move(
            img_folder + file,
            "train/" + file.split(".")[0] + ".jpg",
        )
        shutil.move(
            csv_folder + file.split(".")[0] + ".csv",
            "train/" + file.split(".")[0] + ".csv",
    )
print("TRAINING FOLDER FILLED")
for i in range(25, 26):
    img_folder = f"/kaggle/working/dataset/drafter_{i}/images/"
    csv_folder = f"/kaggle/working/dataset/drafter_{i}/csvs/"

    for file in os.listdir(img_folder):
        if file.split('.')[0] + '.csv' in os.listdir(csv_folder):
            shutil.move(img_folder + file, "test/" + file.split(".")[0] + ".jpg")
            shutil.move(
                csv_folder + file.split(".")[0] + ".csv",
                "test/" + file.split(".")[0] + ".csv",
            )
print("TEST FOLDER FILLED")

TRAINING FOLDER FILLED
TEST FOLDER FILLED


In [7]:
print(len(os.listdir('train')))

4902


# Model Training

In [8]:
classes = set()
boxes = {}
imgs = {}


directory = f'/kaggle/working/train/'
for file in os.listdir(directory):
        if file.endswith(".csv"):
            df = pd.read_csv(directory + file)
            classes.update(df['object'].tolist())

label_mapping = {label: i for i, label in enumerate(classes, start = 1)}

num_classes = len(classes) + 1
print(classes)
gc.collect()

{'probe.current', 'antenna', 'optocoupler', 'capacitor.adjustable', 'capacitor.polarized', 'triac', 'transformer', 'crossover', 'diode.zener', 'unknown', 'crystal', 'microphone', 'switch', 'speaker', 'relay', 'or', 'terminal', 'fuse', 'capacitor.unpolarized', 'integrated_circuit.voltage_regulator', 'diode.thyrector', 'transistor.bjt', 'and', 'motor', 'probe', 'resistor.photo', 'nor', 'operational_amplifier', 'thyristor', 'varistor', 'resistor.adjustable', 'diac', 'resistor', 'diode', 'nand', 'xor', 'transistor.photo', 'voltage.dc', 'transistor.fet', 'lamp', 'voltage.battery', 'vss', 'text', 'probe.voltage', 'operational_amplifier.schmitt_trigger', 'magnetic', 'integrated_circuit', 'gnd', 'inductor', 'junction', 'mechanical', 'inductor.ferrite', 'socket', 'voltage.ac', 'diode.light_emitting', 'not', 'integrated_circuit.ne555', 'optical'}


0

In [9]:
def collate_fn(batch):
    images, targets = zip(*batch)
    images = torch.stack(images)
    return images, targets

class CircuitDataset(Dataset):
    def __init__(self, folder, target_size=(512, 512)):
        self.folder = folder
        self.theight, self.twidth = target_size
        self.names = self.indexes()
    
    def indexes(self):
        names = []
        for filename in os.listdir(self.folder):
            if filename.endswith('.csv'):
                name = filename.split('.')[0]
                names.append(name)
        return names
    
    def __len__(self):
        count = 0
        for path in os.listdir(self.folder):
            if path.endswith('.csv'):
                count += 1
        return count

    def __getitem__(self, idx):
        image_path = os.path.join(self.folder, self.names[idx] + '.jpg')
        boxes_path = os.path.join(self.folder, self.names[idx] + '.csv')
        df = pd.read_csv(boxes_path)
        temp = df[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
        temp2 = [label_mapping[obj] for obj in df['object'].tolist()]
        boxes = {'boxes': t.from_numpy(temp), 'labels': t.tensor(temp2)}
        del temp, temp2
        gc.collect()
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
            
        og_dims = torch.tensor([image.width, image.height, image.width, image.height])
        image = F.resize(image, (self.theight, self.twidth))

        scaled_boxes = (boxes['boxes'] / og_dims) * torch.tensor([self.twidth, self.theight, self.twidth, self.theight])
        # Validate and remove boxes with zero area
        valid_boxes, valid_indices = self.validate_boxes(scaled_boxes)

        image = F.to_tensor(image)
        boxes['boxes'] = valid_boxes.to(torch.float64)
        boxes['labels'] = torch.tensor([boxes['labels'][i] for i in valid_indices], dtype=torch.int64)
        
        del scaled_boxes
        gc.collect()
        return image, boxes

    def validate_boxes(self, boxes):
        widths = boxes[:, 2] - boxes[:, 0]
        heights = boxes[:, 3] - boxes[:, 1]
        valid_indices = (widths > 0) & (heights > 0)
        return boxes[valid_indices], valid_indices.nonzero(as_tuple=True)[0]

train_folder = f'/kaggle/working/train'
test_folder = f'/kaggle/working/test'
train_dataset = CircuitDataset(train_folder)
print("CREATED TRAIN DATASET")
test_dataset = CircuitDataset(test_folder)
print("CREATED TEST DATASET")

train_data_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True, collate_fn = collate_fn, pin_memory = True)
test_data_loader = DataLoader(test_dataset, batch_size = 16, shuffle = True, collate_fn = collate_fn, pin_memory = True)
model = models.detection.fasterrcnn_resnet50_fpn_v2(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

criterion = nn.SmoothL1Loss()

optimizer = optim.Adam(model.parameters(), lr=1e-5, amsgrad = True)
if torch.cuda.is_available():
    device = torch.device('cuda')
    model = nn.DataParallel(model)
    model.to(device)
else:
    device = torch.device('cpu')
    model.to(device)

CREATED TRAIN DATASET
CREATED TEST DATASET


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:01<00:00, 89.7MB/s]


In [10]:
def print_gpu_memory():
    if torch.cuda.is_available():
        print("GPU Memory:")
        for i in range(torch.cuda.device_count()):
            t = torch.cuda.get_device_properties(i).total_memory
            r = torch.cuda.memory_reserved(i)
            a = torch.cuda.memory_allocated(i)
            f = r-a 
            print(f"  GPU {i}: Total = {t/(1024**3):.2f} GB, Reserved = {r/(1024**3):.2f} GB, Allocated = {a/(1024**3):.2f} GB, Free (within reserved) = {f/(1024**3):.2f} GB")
    else:
        print("No CUDA-capable device is detected")


def print_system_memory():
    memory = psutil.virtual_memory()
    total = memory.total / (1024**3)
    available = memory.available / (1024**3)
    print(f"System Memory: Total = {total:.2f} GB, Available = {available:.2f} GB")

In [11]:
print("TRAINING MODEL")
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    running_loss = None
    for images, targets in train_data_loader:
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.sum().backward()
        optimizer.step()
        if running_loss is None:
            running_loss = losses.sum().cpu().detach().numpy()
        else: 
            running_loss += losses.sum().cpu().detach().numpy()
        del losses, loss_dict, images, targets
        torch.cuda.empty_cache() ##This is to make sure it does not crash
        gc.collect()
    print_gpu_memory()
    print_system_memory()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_data_loader)}')
print("MODEL TRAINED")
    
torch.save(model.state_dict(), 'faster_rcnn_model.pth')

TRAINING MODEL
GPU Memory:
  GPU 0: Total = 14.75 GB, Reserved = 1.77 GB, Allocated = 0.84 GB, Free (within reserved) = 0.93 GB
  GPU 1: Total = 14.75 GB, Reserved = 0.84 GB, Allocated = 0.02 GB, Free (within reserved) = 0.83 GB
System Memory: Total = 31.36 GB, Available = 28.39 GB
Epoch [1/10], Loss: 6.687018089451741
GPU Memory:
  GPU 0: Total = 14.75 GB, Reserved = 1.69 GB, Allocated = 0.84 GB, Free (within reserved) = 0.85 GB
  GPU 1: Total = 14.75 GB, Reserved = 0.84 GB, Allocated = 0.02 GB, Free (within reserved) = 0.83 GB
System Memory: Total = 31.36 GB, Available = 28.38 GB
Epoch [2/10], Loss: 3.3403911503780175
GPU Memory:
  GPU 0: Total = 14.75 GB, Reserved = 1.80 GB, Allocated = 0.84 GB, Free (within reserved) = 0.96 GB
  GPU 1: Total = 14.75 GB, Reserved = 0.84 GB, Allocated = 0.02 GB, Free (within reserved) = 0.83 GB
System Memory: Total = 31.36 GB, Available = 28.34 GB
Epoch [3/10], Loss: 3.017650898429466
GPU Memory:
  GPU 0: Total = 14.75 GB, Reserved = 2.14 GB, Allocat

In [12]:
print("SAVED MODEL")

SAVED MODEL


In [13]:
shutil.rmtree('dataset')
shutil.rmtree('train')
shutil.rmtree('test')
print("NOTEBOOK FINISHED")

NOTEBOOK FINISHED
